In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import Descriptors
import py_rdl
import RA

# Cremer Pople Puckering Parameter

In [ ]:
# Five
five = pd.read_csv("data/five.csv")

In [ ]:
monocyclic = []
polycyclic = []
for idx, item in enumerate(five[["A0","A1","A2","A3","A4"]].values):
    if list(item)==["M","M","M","M","M"]:
        monocyclic.append(idx)
    else:
        polycyclic.append(idx)
        
mono_five = five.iloc[monocyclic].reset_index(drop=True)
poly_five = five.iloc[polycyclic].reset_index(drop=True)

mono_single = []
mono_one = []
mono_two = []
for idx, item in enumerate(mono_five[["B0","B1","B2","B3","B4"]].values):
    check = [x==1.5 or x==2.0 for x in item]
    if sum(check)==0:
        mono_single.append(idx)
    elif sum(check)==1:
        mono_one.append(idx)
    else:
        mono_two.append(idx)

poly_single = []
poly_one = []
poly_two = []
for idx, item in enumerate(poly_five[["B0","B1","B2","B3","B4"]].values):
    check = [x==1.5 or x==2.0 for x in list(item)]
    if sum(check)==0:
        poly_single.append(idx)
    elif sum(check)==1:
        poly_one.append(idx)
    else:
        poly_two.append(idx)

plt.hist(mono_five.iloc[mono_single]["q2"],bins=50)
plt.ylabel("Count",size=15)
plt.xlabel(r"$q_{2}$",size=15)
plt.xlim(0,1)
plt.ylim(0,1200)
plt.savefig("Fig/Macrocycle/mono_single_five_q2.pdf",dpi=150,bbox_inches="tight")
plt.show()

plt.hist(poly_five.iloc[poly_single]["q2"],bins=50)
plt.ylabel("Count",size=15)
plt.xlabel(r"$q_{2}$",size=15)
plt.xlim(0,1)
plt.ylim(0,2500)
plt.savefig("Fig/Macrocycle/poly_single_five_q2.pdf",dpi=150,bbox_inches="tight")
plt.show()

plt.hist(mono_five.iloc[mono_single]["phi2"],bins=50)
plt.ylabel("Count",size=15)
plt.xlabel(r"$\phi_{2}$",size=15)
plt.xlim(-np.pi,np.pi)
plt.xticks([-np.pi,-np.pi/2,0,np.pi/2,np.pi],
           labels=[r"$-\pi$",r"$-\pi/2$",r"0",r"$\pi/2$",r"$\pi$"])
plt.ylim(0,600)
plt.savefig("Fig/Macrocycle/mono_single_five_phi2.pdf",dpi=150,bbox_inches="tight")
plt.show()

plt.hist(poly_five.iloc[poly_single]["phi2"],bins=50)
plt.ylabel("Count",size=15)
plt.xlabel(r"$\phi_{2}$",size=15)
plt.xlim(-np.pi,np.pi)
plt.xticks([-np.pi,-np.pi/2,0,np.pi/2,np.pi],
           labels=[r"$-\pi$",r"$-\pi/2$",r"0",r"$\pi/2$",r"$\pi$"])
plt.ylim(0,1400)
plt.savefig("Fig/Macrocycle/poly_single_five_phi2.pdf",dpi=150,bbox_inches="tight")
plt.show()

plt.hist(mono_five.iloc[mono_one]["q2"],bins=50)
plt.ylabel("Count",size=15)
plt.xlabel(r"$q_{2}$",size=15)
plt.xlim(0,1)
plt.ylim(0,1200)
plt.savefig("Fig/Macrocycle/mono_one_five_q2.pdf",dpi=150,bbox_inches="tight")
plt.show()

plt.hist(poly_five.iloc[poly_one]["q2"],bins=50)
plt.ylabel("Count",size=15)
plt.xlabel(r"$q_{2}$",size=15)
plt.xlim(0,1)
plt.ylim(0,7000)
plt.savefig("Fig/Macrocycle/poly_one_five_q2.pdf",dpi=150,bbox_inches="tight")
plt.show()

plt.hist(mono_five.iloc[mono_one]["phi2"],bins=50)
plt.ylabel("Count",size=15)
plt.xlabel(r"$\phi_{2}$",size=15)
plt.xlim(-np.pi,np.pi)
plt.xticks([-np.pi,-np.pi/2,0,np.pi/2,np.pi],
           labels=[r"$-\pi$",r"$-\pi/2$",r"0",r"$\pi/2$",r"$\pi$"])
plt.ylim(0,350)
plt.savefig("Fig/Macrocycle/mono_one_five_phi2.pdf",dpi=150,bbox_inches="tight")
plt.show()

plt.hist(poly_five.iloc[poly_one]["phi2"],bins=50)
plt.ylabel("Count",size=15)
plt.xlabel(r"$\phi_{2}$",size=15)
plt.xlim(-np.pi,np.pi)
plt.xticks([-np.pi,-np.pi/2,0,np.pi/2,np.pi],
           labels=[r"$-\pi$",r"$-\pi/2$",r"0",r"$\pi/2$",r"$\pi$"])
plt.ylim(0,3000)
plt.savefig("Fig/Macrocycle/poly_one_five_phi2.pdf",dpi=150,bbox_inches="tight")
plt.show()

plt.hist(mono_five.iloc[mono_two]["q2"],bins=50)
plt.ylabel("Count",size=15)
plt.xlabel(r"$q_{2}$",size=15)
plt.xlim(0,1)
plt.ylim(0,15000)
plt.savefig("Fig/Macrocycle/mono_two_five_q2.pdf",dpi=150,bbox_inches="tight")
plt.show()

plt.hist(poly_five.iloc[poly_two]["q2"],bins=50)
plt.ylabel("Count",size=15)
plt.xlabel(r"$q_{2}$",size=15)
plt.xlim(0,1)
plt.ylim(0,15000)
plt.savefig("Fig/Macrocycle/poly_two_five_q2.pdf",dpi=150,bbox_inches="tight")
plt.show()

plt.hist(mono_five.iloc[mono_two]["phi2"],bins=50)
plt.ylabel("Count",size=15)
plt.xlabel(r"$\phi_{2}$",size=15)
plt.xlim(-np.pi,np.pi)
plt.xticks([-np.pi,-np.pi/2,0,np.pi/2,np.pi],
           labels=[r"$-\pi$",r"$-\pi/2$",r"0",r"$\pi/2$",r"$\pi$"])
plt.ylim(0,450)
plt.savefig("Fig/Macrocycle/mono_two_five_phi2.pdf",dpi=150,bbox_inches="tight")
plt.show()

plt.hist(poly_five.iloc[poly_two]["phi2"],bins=50)
plt.ylabel("Count",size=15)
plt.xlabel(r"$\phi_{2}$",size=15)
plt.xlim(-np.pi,np.pi)
plt.xticks([-np.pi,-np.pi/2,0,np.pi/2,np.pi],
           labels=[r"$-\pi$",r"$-\pi/2$",r"0",r"$\pi/2$",r"$\pi$"])
plt.ylim(0,1300)
plt.savefig("Fig/Macrocycle/poly_two_five_phi2.pdf",dpi=150,bbox_inches="tight")
plt.show()

In [ ]:
six = pd.read_csv("six.csv")
monocyclic = []
polycyclic = []
for idx, item in enumerate(six[["A0","A1","A2","A3","A4","A5"]].values):
    if list(item)==["M","M","M","M","M","M"]:
        monocyclic.append(idx)
    else:
        polycyclic.append(idx)
        
mono_six = six.iloc[monocyclic].reset_index(drop=True)
poly_six = six.iloc[polycyclic].reset_index(drop=True)

mono_single = []
mono_one = []
mono_two = []
for idx, item in enumerate(mono_six[["B0","B1","B2","B3","B4","B5"]].values):
    check = [x==1.5 or x==2.0 for x in item]
    if sum(check)==0:
        mono_single.append(idx)
    elif sum(check)==1:
        mono_one.append(idx)
    else:
        mono_two.append(idx)

poly_single = []
poly_one = []
poly_two = []
for idx, item in enumerate(poly_six[["B0","B1","B2","B3","B4","B5"]].values):
    check = [x==1.5 or x==2.0 for x in list(item)]
    if sum(check)==0:
        poly_single.append(idx)
    elif sum(check)==1:
        poly_one.append(idx)
    else:
        poly_two.append(idx)

In [ ]:
tidx = []
idx = []
eidx = []
for i, item in enumerate(five[["B0","B1","B2","B3","B4"]].values):
    if sum(item)!=7.5:
        tidx.append(i)
        
aliphatic = five.iloc[tidx].reset_index()
for i, item in enumerate(aliphatic[["A0","A1","A2","A3","A4"]].values):
    if list(item)==["M","M","M","M","M"]:
        idx.append(i)
    else:
        eidx.append(i)

In [ ]:
mono = aliphatic.iloc[idx]
other = aliphatic.iloc[eidx]

In [ ]:
a1 = mono[mono[["B0","B1","B2","B3","B4"]].sum(axis=1)==6.0]
a2 = other[other[["B0","B1","B2","B3","B4"]].sum(axis=1)==6.0]
a3 = mono[mono[["B0","B1","B2","B3","B4"]].sum(axis=1)==5.5]
a4 = other[other[["B0","B1","B2","B3","B4"]].sum(axis=1)==7]
a5 = other[other[["B0","B1","B2","B3","B4"]].sum(axis=1)==6.5]

In [ ]:
#plt.scatter(a1["q2"],a1["phi2"],s=5,alpha=.5)
#plt.scatter(a2["q2"],a2["phi2"],s=5,alpha=.5)
plt.scatter(a4["q2"],a4["phi2"],s=5,alpha=.5)
plt.scatter(a5["q2"],a5["phi2"],s=5,alpha=.5)
plt.show()

In [ ]:
six = pd.read_csv("data/six.csv")

In [ ]:
t1 = []
t2 = []
for idx, item in enumerate(six[["A0","A1","A2","A3","A4","A5"]].values):
    if list(item)==["M","M","M","M","M","M"]:
        t1.append(idx)
    else:
        t2.append(idx)

In [ ]:
six1 = six.iloc[t1].reset_index()
six2 = six.iloc[t2].reset_index()
a1 = []
a2 = []
a3 = []
for idx, item in enumerate(six1[["B0","B1","B2","B3","B4","B5"]].values.sum(axis=1).tolist()):
    if item==6.0:
        a1.append(idx)
    elif item==7.0 or item==6.5:
        a2.append(idx)
    else:
        a3.append(idx)
        
b1 = []
b2 = []
b3 = []
for idx, item in enumerate(six2[["B0","B1","B2","B3","B4","B5"]].values.sum(axis=1).tolist()):
    if item==6.0:
        b1.append(idx)
    elif item==7.0 or item==6.5:
        b2.append(idx)
    else:
        b3.append(idx)

In [ ]:
g1 = sns.pairplot(six1.iloc[a2][["q3","phi2"]], plot_kws=dict(s=15),diag_kws=dict(bins=50))
g1.axes[0,0].set_ylabel(r"$q_{3}$",rotation=0,size=15)
g1.axes[1,0].set_xlabel(r"$q_{3}$",rotation=0,size=15)
g1.axes[1,0].set_ylabel(r"$\phi_{2}$",rotation=0,size=15)
g1.axes[1,1].set_xlabel(r"$\phi_{2}$",rotation=0,size=15)
g1.axes[0,0].set_xlim(-0.6,0.6)
g1.axes[0,0].set_xticks([-0.6,-0.4,-0.2,0,0.2,0.4,0.6])
g1.axes[0,0].set_ylim(-0.6,0.6)
g1.axes[0,1].set_ylim(-0.6,0.6)
g1.axes[1,0].set_yticks([-0.6,-0.4,-0.2,0,0.2,0.4,0.6])
g1.axes[0,1].set_xlim(-np.pi,np.pi)
g1.axes[1,0].set_xlim(-0.6,0.6)
g1.axes[1,0].set_ylim(-np.pi,np.pi)
g1.axes[1,0].set_xticks([-0.6,-0.4,-0.2,0,0.2,0.4,0.6])
g1.axes[1,0].set_yticks([-np.pi,-np.pi/2,0,np.pi/2,np.pi])
g1.axes[1,0].set_yticklabels([r"-$\pi$",r"-$\pi/2$","0",r"$\pi/2$","$\pi$"])
g1.axes[1,1].set_xlim(-np.pi,np.pi)
g1.axes[1,1].set_ylim(-np.pi,np.pi)
g1.axes[1,1].set_xticks([-np.pi,-np.pi/2,0,np.pi/2,np.pi])
g1.axes[1,1].set_xticklabels([r"-$\pi$",r"-$\pi/2$","0",r"$\pi/2$","$\pi$"])
g1.axes[0,1].set_visible(False)
plt.savefig("Fig/Macrocycle/six_one_M_db.pdf",dpi=150, bbox_inches="tight")
plt.show()

In [ ]:
g1 = sns.pairplot(six2.iloc[b2][["q3","phi2"]], plot_kws=dict(s=15),diag_kws=dict(bins=50))
g1.axes[0,0].set_ylabel(r"$q_{3}$",rotation=0,size=15)
g1.axes[1,0].set_xlabel(r"$q_{3}$",rotation=0,size=15)
g1.axes[1,0].set_ylabel(r"$\phi_{2}$",rotation=0,size=15)
g1.axes[1,1].set_xlabel(r"$\phi_{2}$",rotation=0,size=15)
g1.axes[0,0].set_xlim(-0.6,0.6)
g1.axes[0,0].set_xticks([-0.6,-0.4,-0.2,0,0.2,0.4,0.6])
g1.axes[0,0].set_ylim(-0.6,0.6)
g1.axes[0,1].set_ylim(-0.6,0.6)
g1.axes[1,0].set_yticks([-0.6,-0.4,-0.2,0,0.2,0.4,0.6])
g1.axes[0,1].set_xlim(-np.pi,np.pi)
g1.axes[1,0].set_xlim(-0.6,0.6)
g1.axes[1,0].set_ylim(-np.pi,np.pi)
g1.axes[1,0].set_xticks([-0.6,-0.4,-0.2,0,0.2,0.4,0.6])
g1.axes[1,0].set_yticks([-np.pi,-np.pi/2,0,np.pi/2,np.pi])
g1.axes[1,0].set_yticklabels([r"-$\pi$",r"-$\pi/2$","0",r"$\pi/2$","$\pi$"])
g1.axes[1,1].set_xlim(-np.pi,np.pi)
g1.axes[1,1].set_ylim(-np.pi,np.pi)
g1.axes[1,1].set_xticks([-np.pi,-np.pi/2,0,np.pi/2,np.pi])
g1.axes[1,1].set_xticklabels([r"-$\pi$",r"-$\pi/2$","0",r"$\pi/2$","$\pi$"])
g1.axes[0,1].set_visible(False)
plt.savefig("Fig/Macrocycle/six_one_BP_db.pdf",dpi=150, bbox_inches="tight")
plt.show()

In [ ]:
qq1 = six1.iloc[a2][np.abs(six1.iloc[a2]["q3"])<0.03].reset_index()
qq2 = six2.iloc[b2][np.abs(six2.iloc[b2]["q3"])<0.03].reset_index()

In [ ]:
q = six2.iloc[b2].reset_index()

In [ ]:
k = []
for item in q[["B0","B1","B2","B3","B4","B5"]].values:
    k.append(sum([x==1.5 for x in item])!=2)
     

In [ ]:
g1 = sns.pairplot(q.iloc[k][["q3","phi2"]],plot_kws=dict(s=15), diag_kws=dict(bins=50))
g1.axes[0,0].set_ylabel(r"$q_{3}$",rotation=0,size=15)
g1.axes[1,0].set_xlabel(r"$q_{3}$",rotation=0,size=15)
g1.axes[1,0].set_ylabel(r"$\phi_{2}$",rotation=0,size=15)
g1.axes[1,1].set_xlabel(r"$\phi_{2}$",rotation=0,size=15)
g1.axes[0,0].set_xlim(-0.6,0.6)
g1.axes[0,0].set_xticks([-0.6,-0.4,-0.2,0,0.2,0.4,0.6])
g1.axes[0,0].set_ylim(-0.6,0.6)
g1.axes[0,1].set_ylim(-0.6,0.6)
g1.axes[1,0].set_yticks([-0.6,-0.4,-0.2,0,0.2,0.4,0.6])
g1.axes[0,1].set_xlim(-np.pi,np.pi)
g1.axes[1,0].set_xlim(-0.6,0.6)
g1.axes[1,0].set_ylim(-np.pi,np.pi)
g1.axes[1,0].set_xticks([-0.6,-0.4,-0.2,0,0.2,0.4,0.6])
g1.axes[1,0].set_yticks([-np.pi,-np.pi/2,0,np.pi/2,np.pi])
g1.axes[1,0].set_yticklabels([r"-$\pi$",r"-$\pi/2$","0",r"$\pi/2$","$\pi$"])
g1.axes[1,1].set_xlim(-np.pi,np.pi)
g1.axes[1,1].set_ylim(-np.pi,np.pi)
g1.axes[1,1].set_xticks([-np.pi,-np.pi/2,0,np.pi/2,np.pi])
g1.axes[1,1].set_xticklabels([r"-$\pi$",r"-$\pi/2$","0",r"$\pi/2$","$\pi$"])
g1.axes[0,1].set_visible(False)
plt.show()

In [ ]:
ss = []
for idx, item in enumerate(qq[["B0","B1","B2","B3","B4","B5"]].values):
    if sum([x==1.5 for x in item])!=2:
        ss.append(idx)
        

In [ ]:
sns.pairplot(qq.iloc[ss][["q2","q3","phi2"]])

In [ ]:
qq.iloc[ss]

In [ ]:
mol = Chem.MolFromSmiles("C1CCCCC12CCCCCC2")

In [ ]:
RA.GetRingElement(mol, [5,4,3,2,1])

In [ ]:
ringatom = Chem.MolFromSmarts("[r]")

In [ ]:
def GetRingAtomType(mol, ringidx):
    """
    Input:
    
    mol: rdMol
    
    ringidx: list
    
    Return:
    
    outcome: list
    
    """
    ringatom = Chem.MolFromSmarts("[R]")
    monocyclicatom = Chem.MolFromSmarts("[R1]")
    bicyclicatom = Chem.MolFromSmarts("[R2]")
    ringatom_match = [a for x in mol.GetSubstructMatches(ringatom) for a in x]
    monocyclic_match = [a for x in mol.GetSubstructMatches(monocyclicatom) for a in x]
    bicyclic_match = [a for x in mol.GetSubstructMatches(bicyclicatom) for a in x]
    outcome = []
    for i in ringidx:
        if (i in ringatom_match) and (i in monocyclic_match):
            outcome.append("M")
        elif (i in ringatom_match) and (i in bicyclic_match):
            outcome.append("B")
        elif (i in ringatom_match) and (i not in monocyclic_match) and (i not in bicyclic_match):
            outcome.append("P")
        else:
            outcome.append("NA")
    return outcome

In [ ]:
list(range(2,2))

In [ ]:
[item[1] for item in RA.GetRingBonds(mol, range(6))]

In [ ]:
GetRingAtomType(mol, list(range(8)))

In [ ]:
mol.GetSubstructMatches(ringatom)

In [ ]:
mol.GetSubstructMatches(monocyclicatom)

In [ ]:
mol.GetSubstructMatches(bicyclicatom)

In [ ]:
aromatic = []
aliphatic = []
for row in five.iterrows():
    check = [x==1.5 for x in eval(row[1][2])]
    if all(check):
        aromatic.append(row[1])
    else:
        aliphatic.append(row[1])

In [ ]:
s1 = []
s2 = []
s3 = []
for item in aliphatic:
    check = sum(eval(item[2]))
    if check==5.5:
        tmp = eval(item[1])
        s1.append((tmp[0][0],tmp[1][0]))
    elif check==6:
        tmp = eval(item[1])
        s2.append((tmp[0][0],tmp[1][0]))
    elif check==5:
        tmp = eval(item[1])
        s3.append((tmp[0][0],tmp[1][0]))

In [ ]:
pd.DataFrame(s1)[0].hist(bins=50)
pd.DataFrame(s2)[0].hist(bins=50)
pd.DataFrame(s3)[0].hist(bins=50,alpha=.2)

In [ ]:
#pd.DataFrame(s1)[1].hist(bins=50)
#pd.DataFrame(s2)[1].hist(bins=50)
pd.DataFrame(s3)[0].hist(bins=100)

In [ ]:
sns.pairplot(pd.DataFrame(s1),plot_kws=dict(s=5), diag_kws=dict(bins=50))

In [ ]:
sns.pairplot(pd.DataFrame(single),plot_kws=dict(s=5), diag_kws=dict(bins=50))

In [ ]:
single

# Substituents Orientation

In [ ]:
fluorine = pd.read_csv("data/halogen_f.csv")
chlorine = pd.read_csv("data/halogen_cl.csv")
bromine = pd.read_csv("data/halogen_br.csv")
iodine = pd.read_csv("data/halogen_i.csv")
amine0 = pd.read_csv("data/amine_0.csv")
amine1 = pd.read_csv("data/amine_1.csv")
amine2 = pd.read_csv("data/amine_2.csv")
alkoxy = pd.read_csv("data/alkoxy.csv")
hydroxyl = pd.read_csv("data/hydroxyl.csv")
methyl = pd.read_csv("data/methyl.csv")
ketone = pd.read_csv("data/ketone.csv")
carboxylicacid_0 = pd.read_csv("data/carboxylicacid_0.csv")
carboxylicacid_1 = pd.read_csv("data/carboxylicacid_1.csv")
carboxylicacid_2 = pd.read_csv("data/carboxylicacid_2.csv")
carbonyl = pd.read_csv("data/carbonyl.csv")

In [ ]:
# Substituents
aliphatic = [len(aliphatic_fluorine),len(aliphatic_chlorine),len(aliphatic_bromine), len(aliphatic_iodine), 
             len(aliphatic_alkoxy), len(aliphatic_hydroxyl), len(aliphatic_amine0),len(aliphatic_amine1)
             ,len(aliphatic_amine2), len(aliphatic_methyl), len(aliphatic_ketone), len(aliphatic_carbonyl), len(aliphatic_carboxylicacid0),
            len(aliphatic_carboxylicacid1)]
aromatic = [len(aromatic_fluorine),len(aromatic_chlorine),len(aromatic_bromine), len(aromatic_iodine), 
           len(aromatic_alkoxy), len(aromatic_hydroxyl), len(aromatic_amine0), len(aromatic_amine1),
            len(aromatic_amine2), len(aromatic_methyl), len(aromatic_ketone), len(aromatic_carbonyl), len(aromatic_carboxylicacid0),
           len(aromatic_carboxylicacid1)]
types = ["aliphatic"]*len(aliphatic) + ["aromatic"]*len(aromatic)
name = ["Fluorine","Chlorine","Bromine","Iodine","Alkoxy","Hydroxyl","Tertiary Amine","Secondary Amine","Primary Amine", 
        "Methyl", "Ketone", "Carbonyl","Carboxylic Acid", "Carboxylic"]

dataframe = pd.DataFrame({"Types":types,"Count":aliphatic+aromatic, "Name":name*2})

sns.barplot(x="Name",y="Count", data=dataframe, hue="Types")
plt.yscale("log")
plt.xticks(range(len(name)), name, rotation='vertical')
plt.xlabel("Substituents",size=15)
plt.ylabel("Count",size=15)
plt.legend()
plt.show()

In [ ]:
aromatic_fluorine = fluorine[fluorine["aromatic"]==True]
aliphatic_fluorine = fluorine[fluorine["aromatic"]==False]

aromatic_chlorine = chlorine[chlorine["aromatic"]==True]
aliphatic_chlorine = chlorine[chlorine["aromatic"]==False]

aromatic_bromine = bromine[bromine["aromatic"]==True]
aliphatic_bromine = bromine[bromine["aromatic"]==False]

aromatic_iodine = iodine[iodine["aromatic"]==True]
aliphatic_iodine = iodine[iodine["aromatic"]==False]

aromatic_methyl = methyl[methyl["aromatic"]==True]
aliphatic_methyl = methyl[methyl["aromatic"]==False]

aromatic_alkoxy = alkoxy[alkoxy["aromatic"]==True]
aliphatic_alkoxy = alkoxy[alkoxy["aromatic"]==False]

aromatic_hydroxyl = hydroxyl[hydroxyl["aromatic"]==True]
aliphatic_hydroxyl = hydroxyl[hydroxyl["aromatic"]==False]

aromatic_amine0 = amine0[amine0["aromatic"]==True]
aliphatic_amine0 = amine0[amine0["aromatic"]==False]

aromatic_amine1 = amine1[amine1["aromatic"]==True]
aliphatic_amine1 = amine1[amine1["aromatic"]==False]

aromatic_amine2 = amine2[amine2["aromatic"]==True]
aliphatic_amine2 = amine2[amine2["aromatic"]==False]

aromatic_ketone = ketone[ketone["aromatic"]==True]
aliphatic_ketone = ketone[ketone["aromatic"]==False]

aromatic_carbonyl = carbonyl[carbonyl["aromatic"]==True]
aliphatic_carbonyl = carbonyl[carbonyl["aromatic"]==False]

aromatic_carboxylicacid0 = carboxylicacid_0[carboxylicacid_0["aromatic"]==True]
aliphatic_carboxylicacid0 = carboxylicacid_0[carboxylicacid_0["aromatic"]==False]

aromatic_carboxylicacid1 = carboxylicacid_1[carboxylicacid_1["aromatic"]==True]
aliphatic_carboxylicacid1 = carboxylicacid_1[carboxylicacid_1["aromatic"]==False]

In [ ]:
plt.hist(aromatic_fluorine["alpha"],bins=50)
plt.xlim(0,np.pi)
plt.xticks([0,np.pi/4, np.pi/2, 3*np.pi/4,np.pi],labels=[r"0",r"$\pi/4$",r"$\pi/2$",r"$3\pi/4$",r"$\pi$"])
plt.ylim(0,1800)
plt.xlabel(r"$\alpha$",size=15)
plt.ylabel("Count",size=15)
plt.savefig("aromatic_flurine_alpha.pdf",dpi=150,bbox_inches="tight")
plt.show()

plt.hist(aromatic_chlorine["alpha"],bins=50)
plt.xlim(0,np.pi)
plt.xticks([0,np.pi/4, np.pi/2, 3*np.pi/4,np.pi],labels=[r"0",r"$\pi/4$",r"$\pi/2$",r"$3\pi/4$",r"$\pi$"])
plt.ylim(0,6000)
plt.xlabel(r"$\alpha$",size=15)
plt.ylabel("Count",size=15)
plt.savefig("aromatic_chlorine_alpha.pdf",dpi=150,bbox_inches="tight")
plt.show()

plt.hist(aromatic_bromine["alpha"],bins=50)
plt.xlim(0,np.pi)
plt.xticks([0,np.pi/4, np.pi/2, 3*np.pi/4,np.pi],labels=[r"0",r"$\pi/4$",r"$\pi/2$",r"$3\pi/4$",r"$\pi$"])
plt.ylim(0,3000)
plt.xlabel(r"$\alpha$",size=15)
plt.ylabel("Count",size=15)
plt.savefig("aromatic_bromine_alpha.pdf",dpi=150,bbox_inches="tight")
plt.show()

plt.hist(aromatic_iodine["alpha"],bins=50)
plt.xlim(0,np.pi)
plt.xticks([0,np.pi/4, np.pi/2, 3*np.pi/4,np.pi],labels=[r"0",r"$\pi/4$",r"$\pi/2$",r"$3\pi/4$",r"$\pi$"])
plt.ylim(0,600)
plt.xlabel(r"$\alpha$",size=15)
plt.ylabel("Count",size=15)
plt.savefig("aromatic_iodine_alpha.pdf",dpi=150,bbox_inches="tight")
plt.show()

In [ ]:
plt.hist(aromatic_fluorine["beta"],bins=30)
plt.xlim(-np.pi,np.pi)
plt.xticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi],labels=[r"$-\pi$",r"$-\pi/2$",r"$0$",r"$\pi/2$",r"$\pi$"])
plt.ylim(0,1250)
plt.xlabel(r"$\beta$",size=15)
plt.ylabel("Count",size=15)
plt.savefig("aromatic_fluorine_beta.pdf",dpi=150,bbox_inches="tight")
plt.show()

plt.hist(aromatic_chlorine["beta"],bins=30)
plt.xlim(-np.pi,np.pi)
plt.xticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi],labels=[r"$-\pi$",r"$-\pi/2$",r"$0$",r"$\pi/2$",r"$\pi$"])
plt.ylim(0,1200)
plt.xlabel(r"$\beta$",size=15)
plt.ylabel("Count",size=15)
plt.savefig("aromatic_chlorine_beta.pdf",dpi=150,bbox_inches="tight")
plt.show()

plt.hist(aromatic_bromine["beta"],bins=30)
plt.xlim(-np.pi,np.pi)
plt.xticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi],labels=[r"$-\pi$",r"$-\pi/2$",r"$0$",r"$\pi/2$",r"$\pi$"])
plt.ylim(0,1200)
plt.xlabel(r"$\beta$",size=15)
plt.ylabel("Count",size=15)
plt.savefig("aromatic_bromine_beta.pdf",dpi=150,bbox_inches="tight")
plt.show()

plt.hist(aromatic_iodine["beta"],bins=30)
plt.xlim(-np.pi,np.pi)
plt.xticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi],labels=[r"$-\pi$",r"$-\pi/2$",r"$0$",r"$\pi/2$",r"$\pi$"])
plt.ylim(0,500)
plt.xlabel(r"$\beta$",size=15)
plt.ylabel("Count",size=15)
plt.savefig("aromatic_iodine_beta.pdf",dpi=150,bbox_inches="tight")
plt.show()

In [ ]:
plt.hist(aliphatic_fluorine["alpha"],bins=50)
plt.xlim(0,np.pi)
plt.xticks([0,np.pi/4, np.pi/2, 3*np.pi/4,np.pi],labels=[r"0",r"$\pi/4$",r"$\pi/2$",r"$3\pi/4$",r"$\pi$"])
plt.ylim(0,100)
plt.xlabel(r"$\alpha$",size=15)
plt.ylabel("Count",size=15)
plt.savefig("aliphatic_fluorine_alpha.pdf",dpi=150,bbox_inches="tight")
plt.show()

plt.hist(aliphatic_chlorine["alpha"],bins=50)
plt.xlim(0,np.pi)
plt.xticks([0,np.pi/4, np.pi/2, 3*np.pi/4,np.pi],labels=[r"0",r"$\pi/4$",r"$\pi/2$",r"$3\pi/4$",r"$\pi$"])
plt.ylim(0,200)
plt.xlabel(r"$\alpha$",size=15)
plt.ylabel("Count",size=15)
plt.savefig("aliphatic_chlorine_alpha.pdf",dpi=150,bbox_inches="tight")
plt.show()

plt.hist(aliphatic_bromine["alpha"],bins=50)
plt.xlim(0,np.pi)
plt.xticks([0,np.pi/4, np.pi/2, 3*np.pi/4,np.pi],labels=[r"0",r"$\pi/4$",r"$\pi/2$",r"$3\pi/4$",r"$\pi$"])
plt.ylim(0,100)
plt.xlabel(r"$\alpha$",size=15)
plt.ylabel("Count",size=15)
plt.savefig("aliphatic_bromine_alpha.pdf",dpi=150,bbox_inches="tight")
plt.show()

plt.hist(aliphatic_iodine["alpha"],bins=50)
plt.xlim(0,np.pi)
plt.xticks([0,np.pi/4, np.pi/2, 3*np.pi/4,np.pi],labels=[r"0",r"$\pi/4$",r"$\pi/2$",r"$3\pi/4$",r"$\pi$"])
plt.ylim(0,50)
plt.xlabel(r"$\alpha$",size=15)
plt.ylabel("Count",size=15)
plt.savefig("aliphatic_iodine_alpha.pdf",dpi=150,bbox_inches="tight")
plt.show()

In [ ]:
plt.hist(aliphatic_fluorine["beta"],bins=100)
plt.xlim(-np.pi,np.pi)
plt.xticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi],labels=[r"$-\pi$",r"$-\pi/2$",r"$0$",r"$\pi/2$",r"$\pi$"])
plt.ylim(0,500)
plt.xlabel(r"$\beta$",size=15)
plt.ylabel("Count",size=15)
plt.savefig("aliphatic_fluorine_beta.pdf",dpi=150,bbox_inches="tight")
plt.show()

plt.hist(aliphatic_chlorine["beta"],bins=100)
plt.xlim(-np.pi,np.pi)
plt.xticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi],labels=[r"$-\pi$",r"$-\pi/2$",r"$0$",r"$\pi/2$",r"$\pi$"])
plt.ylim(0,400)
plt.xlabel(r"$\beta$",size=15)
plt.ylabel("Count",size=15)
plt.savefig("aliphatic_chlorine_beta.pdf",dpi=150,bbox_inches="tight")
plt.show()

plt.hist(aliphatic_bromine["beta"],bins=100)
plt.xlim(-np.pi,np.pi)
plt.xticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi],labels=[r"-$\pi$",r"$-\pi/2$",r"$0$",r"$\pi/2$",r"$\pi$"])
plt.ylim(0,200)
plt.xlabel(r"$\beta$",size=15)
plt.ylabel("Count",size=15)
plt.savefig("aliphatic_bromine_beta.pdf",dpi=150,bbox_inches="tight")
plt.show()

plt.hist(aliphatic_iodine["beta"],bins=100)
plt.xlim(-np.pi,np.pi)
plt.xticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi],labels=[r"-$\pi$",r"$-\pi/2$",r"$0$",r"$\pi/2$",r"$\pi$"])
plt.ylim(0,50)
plt.xlabel(r"$\beta$",size=15)
plt.ylabel("Count",size=15)
plt.savefig("aliphatic_iodine_beta.pdf",dpi=150,bbox_inches="tight")
plt.show()

# Methyl

In [ ]:
plt.hist(aliphatic_methyl["alpha"],bins=100)
plt.xlim(0,np.pi)
plt.xticks([0,np.pi/4, np.pi/2, 3*np.pi/4,np.pi],labels=[r"0",r"$\pi/4$",r"$\pi/2$",r"$3\pi/4$",r"$\pi$"])
plt.ylim(0,5000)
plt.xlabel(r"$\alpha$",size=15)
plt.ylabel("Count",size=15)
plt.savefig("aliphatic_methyl_alpha.pdf",dpi=150,bbox_inches="tight")
plt.show()

plt.hist(aromatic_methyl["alpha"],bins=100)
plt.xlim(0,np.pi)
plt.xticks([0,np.pi/4, np.pi/2, 3*np.pi/4,np.pi],labels=[r"0",r"$\pi/4$",r"$\pi/2$",r"$3\pi/4$",r"$\pi$"])
plt.ylim(0,25000)
plt.xlabel(r"$\alpha$",size=15)
plt.ylabel("Count",size=15)
plt.savefig("aromatic_methyl_alpha.pdf",dpi=150,bbox_inches="tight")
plt.show()

plt.hist(aliphatic_methyl["beta"],bins=100)
plt.xlim(-np.pi,np.pi)
plt.xticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi],labels=[r"$-\pi$",r"$-\pi/2$",r"$0$",r"$\pi/2$",r"$\pi$"])
plt.ylim(0,18000)
plt.xlabel(r"$\beta$",size=15)
plt.ylabel("Count",size=15)
plt.savefig("aliphatic_methyl_beta.pdf",dpi=150,bbox_inches="tight")
plt.show()

plt.hist(aromatic_methyl["beta"],bins=100)
plt.xlim(-np.pi,np.pi)
plt.xticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi],labels=[r"$-\pi$",r"$-\pi/2$",r"$0$",r"$\pi/2$",r"$\pi$"])
plt.ylim(0,4000)
plt.xlabel(r"$\beta$",size=15)
plt.ylabel("Count",size=15)
plt.savefig("aromatic_methyl_beta.pdf",dpi=150,bbox_inches="tight")
plt.show()

# Alkoxy and Hydroxyl

In [ ]:
plt.hist(aliphatic_alkoxy["alpha"],bins=100)
plt.xlim(0,np.pi)
plt.xticks([0,np.pi/4, np.pi/2, 3*np.pi/4,np.pi],labels=[r"0",r"$\pi/4$",r"$\pi/2$",r"$3\pi/4$",r"$\pi$"])
plt.ylim(0,800)
plt.xlabel(r"$\alpha$",size=15)
plt.ylabel("Count",size=15)
plt.savefig("aliphatic_alkoxy_alpha.pdf",dpi=150,bbox_inches="tight")
plt.show()

plt.hist(aromatic_alkoxy["alpha"],bins=100)
plt.xlim(0,np.pi)
plt.xticks([0,np.pi/4, np.pi/2, 3*np.pi/4,np.pi],labels=[r"0",r"$\pi/4$",r"$\pi/2$",r"$3\pi/4$",r"$\pi$"])
plt.ylim(0,14000)
plt.xlabel(r"$\alpha$",size=15)
plt.ylabel("Count",size=15)
plt.savefig("aromatic_alkoxy_alpha.pdf",dpi=150,bbox_inches="tight")
plt.show()

plt.hist(aliphatic_hydroxyl["alpha"],bins=100)
plt.xlim(0,np.pi)
plt.xticks([0,np.pi/4, np.pi/2, 3*np.pi/4,np.pi],labels=[r"0",r"$\pi/4$",r"$\pi/2$",r"$3\pi/4$",r"$\pi$"])
plt.ylim(0,2000)
plt.xlabel(r"$\alpha$",size=15)
plt.ylabel("Count",size=15)
plt.savefig("aliphatic_hydroxyl_alpha.pdf",dpi=150,bbox_inches="tight")
plt.show()

plt.hist(aromatic_hydroxyl["alpha"],bins=100)
plt.xlim(0,np.pi)
plt.xticks([0,np.pi/4, np.pi/2, 3*np.pi/4,np.pi],labels=[r"0",r"$\pi/4$",r"$\pi/2$",r"$3\pi/4$",r"$\pi$"])
plt.ylim(0,8000)
plt.xlabel(r"$\alpha$",size=15)
plt.ylabel("Count",size=15)
plt.savefig("aromatic_hydroxyl_alpha.pdf",dpi=150,bbox_inches="tight")
plt.show()


In [ ]:
plt.hist(aliphatic_alkoxy["beta"],bins=100)
plt.xlim(-np.pi,np.pi)
plt.xticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi],labels=[r"$-\pi$",r"$-\pi/2$",r"$0$",r"$\pi/2$",r"$\pi$"])
plt.ylim(0,2500)
plt.xlabel(r"$\beta$",size=15)
plt.ylabel("Count",size=15)
plt.savefig("aliphatic_alkoxy_beta.pdf",dpi=150,bbox_inches="tight")
plt.show()

plt.hist(aromatic_alkoxy["beta"],bins=100)
plt.xlim(-np.pi,np.pi)
plt.xticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi],labels=[r"$-\pi$",r"$-\pi/2$",r"$0$",r"$\pi/2$",r"$\pi$"])
plt.ylim(0,4500)
plt.xlabel(r"$\beta$",size=15)
plt.ylabel("Count",size=15)
plt.savefig("aromatic_alkoxy_beta.pdf",dpi=150,bbox_inches="tight")
plt.show()

plt.hist(aliphatic_hydroxyl["beta"],bins=100)
plt.xlim(-np.pi,np.pi)
plt.xticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi],labels=[r"$-\pi$",r"$-\pi/2$",r"$0$",r"$\pi/2$",r"$\pi$"])
plt.ylim(0,13000)
plt.xlabel(r"$\beta$",size=15)
plt.ylabel("Count",size=15)
plt.savefig("aliphatic_hydroxyl_beta.pdf",dpi=150,bbox_inches="tight")
plt.show()

plt.hist(aromatic_hydroxyl["beta"],bins=100)
plt.xlim(-np.pi,np.pi)
plt.xticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi],labels=[r"$-\pi$",r"$-\pi/2$",r"$0$",r"$\pi/2$",r"$\pi$"])
plt.ylim(0,1500)
plt.xlabel(r"$\beta$",size=15)
plt.ylabel("Count",size=15)
plt.savefig("aromatic_hydroxyl_beta.pdf",dpi=150,bbox_inches="tight")
plt.show()

# Amine

In [ ]:
## Aromatic
plt.hist(aromatic_amine0["alpha"],bins=100)
plt.xlim(0,np.pi)
plt.xticks([0,np.pi/4, np.pi/2, 3*np.pi/4,np.pi],labels=[r"0",r"$\pi/4$",r"$\pi/2$",r"$3\pi/4$",r"$\pi$"])
plt.ylim(0,7000)
plt.xlabel(r"$\alpha$",size=15)
plt.ylabel("Count",size=15)
plt.savefig("aromatic_amine0_alpha.pdf",dpi=150,bbox_inches="tight")
plt.show()

plt.hist(aromatic_amine1["alpha"],bins=100)
plt.xlim(0,np.pi)
plt.xticks([0,np.pi/4, np.pi/2, 3*np.pi/4,np.pi],labels=[r"0",r"$\pi/4$",r"$\pi/2$",r"$3\pi/4$",r"$\pi$"])
plt.ylim(0,2000)
plt.xlabel(r"$\alpha$",size=15)
plt.ylabel("Count",size=15)
plt.savefig("aromatic_amine1_alpha.pdf",dpi=150,bbox_inches="tight")
plt.show()

plt.hist(aromatic_amine2["alpha"],bins=100)
plt.xlim(0,np.pi)
plt.xticks([0,np.pi/4, np.pi/2, 3*np.pi/4,np.pi],labels=[r"0",r"$\pi/4$",r"$\pi/2$",r"$3\pi/4$",r"$\pi$"])
plt.ylim(0,150)
plt.xlabel(r"$\alpha$",size=15)
plt.ylabel("Count",size=15)
plt.savefig("aromatic_amine2_alpha.pdf",dpi=150,bbox_inches="tight")
plt.show()


plt.hist(aliphatic_amine0["alpha"],bins=100)
plt.xlim(0,np.pi)
plt.xticks([0,np.pi/4, np.pi/2, 3*np.pi/4,np.pi],labels=[r"0",r"$\pi/4$",r"$\pi/2$",r"$3\pi/4$",r"$\pi$"])
plt.ylim(0,100)
plt.xlabel(r"$\alpha$",size=15)
plt.ylabel("Count",size=15)
plt.savefig("aliphatic_amine0_alpha.pdf",dpi=150,bbox_inches="tight")
plt.show()

plt.hist(aliphatic_amine1["alpha"],bins=100)
plt.xlim(0,np.pi)
plt.xticks([0,np.pi/4, np.pi/2, 3*np.pi/4,np.pi],labels=[r"0",r"$\pi/4$",r"$\pi/2$",r"$3\pi/4$",r"$\pi$"])
plt.ylim(0,250)
plt.xlabel(r"$\alpha$",size=15)
plt.ylabel("Count",size=15)
plt.savefig("aliphatic_amine1_alpha.pdf",dpi=150,bbox_inches="tight")
plt.show()

plt.hist(aliphatic_amine2["alpha"],bins=100)
plt.xlim(0,np.pi)
plt.xticks([0,np.pi/4, np.pi/2, 3*np.pi/4,np.pi],labels=[r"0",r"$\pi/4$",r"$\pi/2$",r"$3\pi/4$",r"$\pi$"])
plt.ylim(0,150)
plt.xlabel(r"$\alpha$",size=15)
plt.ylabel("Count",size=15)
plt.savefig("aliphatic_amine2_alpha.pdf",dpi=150,bbox_inches="tight")
plt.show()



In [ ]:
## Aromatic
plt.hist(aromatic_amine0["beta"],bins=100)
plt.xlim(-np.pi,np.pi)
plt.xticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi],labels=[r"$-\pi$",r"$-\pi/2$",r"$0$",r"$\pi/2$",r"$\pi$"])
plt.ylim(0,5500)
plt.xlabel(r"$\beta$",size=15)
plt.ylabel("Count",size=15)
plt.savefig("aromatic_amine0_beta.pdf",dpi=150,bbox_inches="tight")
plt.show()

plt.hist(aromatic_amine1["beta"],bins=100)
plt.xlim(-np.pi,np.pi)
plt.xticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi],labels=[r"$-\pi$",r"$-\pi/2$",r"$0$",r"$\pi/2$",r"$\pi$"])
plt.ylim(0,600)
plt.xlabel(r"$\beta$",size=15)
plt.ylabel("Count",size=15)
plt.savefig("aromatic_amine1_beta.pdf",dpi=150,bbox_inches="tight")
plt.show()

plt.hist(aromatic_amine2["beta"],bins=100)
plt.xlim(-np.pi,np.pi)
plt.xticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi],labels=[r"$-\pi$",r"$-\pi/2$",r"$0$",r"$\pi/2$",r"$\pi$"])
plt.ylim(0,300)
plt.xlabel(r"$\beta$",size=15)
plt.ylabel("Count",size=15)
plt.savefig("aromatic_amine2_beta.pdf",dpi=150,bbox_inches="tight")
plt.show()

## Aliphatic
plt.hist(aliphatic_amine0["beta"],bins=100)
plt.xlim(-np.pi,np.pi)
plt.xticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi],labels=[r"$-\pi$",r"$-\pi/2$",r"$0$",r"$\pi/2$",r"$\pi$"])
plt.ylim(0,600)
plt.xlabel(r"$\beta$",size=15)
plt.ylabel("Count",size=15)
plt.savefig("aliphatic_amine0_beta.pdf",dpi=150,bbox_inches="tight")
plt.show()

plt.hist(aliphatic_amine1["beta"],bins=100)
plt.xlim(-np.pi,np.pi)
plt.xticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi],labels=[r"$-\pi$",r"$-\pi/2$",r"$0$",r"$\pi/2$",r"$\pi$"])
plt.ylim(0,1500)
plt.xlabel(r"$\beta$",size=15)
plt.ylabel("Count",size=15)
plt.savefig("aliphatic_amine1_beta.pdf",dpi=150,bbox_inches="tight")
plt.show()

plt.hist(aliphatic_amine2["beta"],bins=100)
plt.xlim(-np.pi,np.pi)
plt.xticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi],labels=[r"$-\pi$",r"$-\pi/2$",r"$0$",r"$\pi/2$",r"$\pi$"])
plt.ylim(0,300)
plt.xlabel(r"$\beta$",size=15)
plt.ylabel("Count",size=15)
plt.savefig("aliphatic_amine2_beta.pdf",dpi=150,bbox_inches="tight")
plt.show()

# Ketone

In [ ]:
plt.hist(aliphatic_ketone["alpha"],bins=100)
plt.xlim(0,np.pi)
plt.xticks([0,np.pi/4, np.pi/2, 3*np.pi/4,np.pi],labels=[r"0",r"$\pi/4$",r"$\pi/2$",r"$3\pi/4$",r"$\pi$"])
plt.ylim(0,800)
plt.xlabel(r"$\alpha$",size=15)
plt.ylabel("Count",size=15)
plt.savefig("aliphatic_ketone_alpha.pdf",dpi=150,bbox_inches="tight")
plt.show()

plt.hist(aromatic_ketone["alpha"],bins=100)
plt.xlim(0,np.pi)
plt.xticks([0,np.pi/4, np.pi/2, 3*np.pi/4,np.pi],labels=[r"0",r"$\pi/4$",r"$\pi/2$",r"$3\pi/4$",r"$\pi$"])
plt.ylim(0,9000)
plt.xlabel(r"$\alpha$",size=15)
plt.ylabel("Count",size=15)
plt.savefig("aromatic_ketone_alpha.pdf",dpi=150,bbox_inches="tight")
plt.show()

In [ ]:
plt.hist(aliphatic_ketone["beta"],bins=100)
plt.xlim(-np.pi,np.pi)
plt.xticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi],labels=[r"$-\pi$",r"$-\pi/2$",r"$0$",r"$\pi/2$",r"$\pi$"])
plt.ylim(0,5500)
plt.xlabel(r"$\beta$",size=15)
plt.ylabel("Count",size=15)
plt.savefig("aliphatic_ketone_beta.pdf",dpi=150,bbox_inches="tight")
plt.show()

plt.hist(aromatic_ketone["beta"],bins=100)
plt.xlim(-np.pi,np.pi)
plt.xticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi],labels=[r"$-\pi$",r"$-\pi/2$",r"$0$",r"$\pi/2$",r"$\pi$"])
plt.ylim(0,1500)
plt.xlabel(r"$\beta$",size=15)
plt.ylabel("Count",size=15)
plt.savefig("aromatic_ketone_beta.pdf",dpi=150,bbox_inches="tight")
plt.show()

# Carbonyl

In [ ]:
plt.hist(aliphatic_carbonyl["alpha"],bins=100)
plt.xlim(0,np.pi)
plt.xticks([0,np.pi/4, np.pi/2, 3*np.pi/4,np.pi],labels=[r"0",r"$\pi/4$",r"$\pi/2$",r"$3\pi/4$",r"$\pi$"])
plt.ylim(0,10000)
plt.xlabel(r"$\alpha$",size=15)
plt.ylabel("Count",size=15)
plt.savefig("aliphatic_carbonyl_alpha.pdf",dpi=150,bbox_inches="tight")
plt.show()

In [ ]:
plt.hist(aliphatic_carbonyl["beta"],bins=100)
plt.xlim(-np.pi,np.pi)
plt.xticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi],labels=[r"$-\pi$",r"$-\pi/2$",r"$0$",r"$\pi/2$",r"$\pi$"])
plt.ylim(0,35000)
plt.xlabel(r"$\beta$",size=15)
plt.ylabel("Count",size=15)
#plt.savefig("aliphatic_carbonyl_beta.pdf",dpi=150,bbox_inches="tight")
plt.show()


In [ ]:
alkoxy = Chem.SDMolSupplier("molecule/f.sdf")
alkoxy_smarts = "[R][F&!R]"
c1 = []
for idx, mol in enumerate(alkoxy):
    bonds = []
    for bond in mol.GetBonds():
        bonds.append((bond.GetBeginAtom().GetIdx(),bond.GetEndAtom().GetIdx()))
    data = py_rdl.Calculator.get_calculated_result(bonds)
    for urf in data.urfs:
        rcs = data.get_relevant_cycles_for_urf(urf)
        for rc in rcs:
            if (len(list(rc.nodes))<20) and (len(list(rc.nodes))>4):
                ringloop = RA.Rearrangement(mol, list(rc.nodes))
                coord = np.array([mol.GetConformer().GetAtomPosition(atom) for atom in ringloop])
                ox = RA.GetSubstituent(mol, list(rc.nodes), alkoxy_smarts)
                if any(ox):
                    for i in ox:
                        ccoord = RA.Translate(coord)
                        cptmp = RA.GetRingPuckerCoords(ccoord)
                        ohpos = RA.GetRingSubstituentPosition(mol, ringloop, i)
                        b = RA.GetRingBonds(mol, ringloop)
                        c1.append((idx,cptmp,ohpos,b))


In [ ]:
RA.GetRingBonds(mol)

In [ ]:
mol = Chem.MolFromSmiles("c1ccccc1")

In [ ]:
RA.GetRingBonds(mol,list(range(6)))